<a href="https://colab.research.google.com/github/MosaicGroupCMU/African-MRS-Tutorials/blob/main/Google-Colab/5_QE_Vibrational_Modes_H2_Easy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install Quantum ESPRESSO and ASE

**Run the cell below at the start of today's class to install QE and ASE.**

NOTE: Quantum ESPRESSO installation takes ~8 minutes. ASE installation takes ~3 minutes.

In [ ]:
# eliminate text output during installation
%%capture

# clone Quantum ESPRESSO git file - may take ~5 mins
# (the exclamation mark means that the command is run under Linux)
!git clone https://github.com/QEF/q-e.git

# install mathematical libraries to peform fast Fourier transforms
!apt-get install -y libfftw3-3 libfftw3-dev libfftw3-doc

# run configuration
%cd q-e
!DFLAGS='-D__FFTW3 ' FFT_LIBS='-lfftw3' ./configure
# install executables required for this workbook - may take ~3 mins
!make pw
!make ph
!make dynmat
!make matdyn
!make q2r
!make plotband

# install the Atomic simulation environment - may take ~3 mins
# ! apt install ase
! pip install git+https://gitlab.com/ase/ase

---
# Vibrational modes of molecules

In this tutorial, you will lean how to calculate the vibrational frequencies of molecules from first principles, using molecular hydrogen as an example.

Contributors: [Lorenzo Bastonero](https://github.com/bastonero), [Jessica Wen](https://github.com/JessicaWen-PhD)

This easy workbook is designed for people who are unfamiliar with Python and terminal commands.

If this notebook is too easy for you, there is a [medium-level workbook]() designed for people who have done Python and worked with terminal commands before, but are unfamiliar with Quantum ESPRESSO; if you have already dabbled in some Quantum ESPRESSO, you can try the [hard version]() of the workbook.

# The harmonic approximation

Atoms in molecules and solids vibrate around their _equilibrium_ positions.

![n2harmonicoscillation.png](https://upload.wikimedia.org/wikipedia/commons/a/aa/N2ground.png)

# Setup

This part installs libraries for numerical calculations and plotting.

In [ ]:
import os
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt

This part uploads the pseudopotentials needed for our calculations.

In [ ]:
# eliminate text output during installation
%%capture

# create pseudopotentials folder and navigate into it
%mkdir -p /content/q-e/pseudopotentials
%cd /content/q-e/pseudopotentials/

# download the H pseudopotential file from the GitHub
! wget https://raw.githubusercontent.com/MosaicGroupCMU/African-MRS-Tutorials/refs/heads/main/Google-Colab/H.UPF

# navigate to main directory named '/content/'
%cd ../..

# clean up some files
! rm -rf sample_data

## $H_2$ vibrational mode: finite differences of forces

### Step 1: find the _equilibrium_ geometry.

In [ ]:
# Importing required libraries
from ase import Atoms
from ase.visualize import view

# Step 1: Define H2 molecule with bond length
# H-H bond length is given as 0.74 angstroms in the literature.
bond_length = _________

# Create an H2 molecule with two hydrogen atoms
h2_molecule = Atoms('H2', positions=[[_________, _________, _________], [_________, _________, _________]])

# Step 2: Visualize the molecule
view(_________, viewer='x3d')
# hint: try rotating the molecule and zooming in and out!

In [ ]:
# Optional output: Print details of the H2 molecule
print("H2 Molecule Information:")
print("Atomic Positions (in Angstrom):")
print(h2_molecule.get_positions())
print("Atomic Numbers:", h2_molecule.get_atomic_numbers())
print("Bond Length:", bond_length)

Fill out the base input file (remember definitions can be found [here](https://www.ecosia.org/search?q=quantum+espresso+input+file&addon=chrome&addonversion=6.0.4&method=topbar&addon=opensearch)).

In [ ]:
# create calculation folder and navigate into it
%mkdir -p /content/hydrogen
%cd /content/hydrogen/

# create input and write it into the file h2.scf.in
qe_input = """
&control
  prefix='h2',
  calculation='_________',
  verbosity='high',
  pseudo_dir = '_________',
  outdir='/content/_________/'
  tstress=false,
  tprnfor=false,
  restart_mode='from_scratch',
/
&system
  occupations='smearing',
  smearing='_________', ! try using the Marzari-Vanderbilt-DeVita-Payne cold smearing (see PRL 82, 3296 (1999))
  degauss=0.01,
  ecutwfc = _________,
  ecutrho = _________,
  ibrav = 1,
  celldm(1) = 20,
  nat = _________,
  ntyp = _________,
/
&electrons
  conv_thr = 1e-6,
/
ATOMIC_SPECIES
 _________  1.00784  _________
ATOMIC_POSITIONS (angstrom)
 _________ _________ _________ _________
 _________ _________ _________ _________
K_POINTS (automatic)
  1 1 1 0 0 0
"""

with open("_________", "w") as f:
    f.write(_________)

# print the content of the input file (under Linux)
! cat _________

Calculate the potential energy surface for different distances of the atoms. We use a similar process to exercise 1 (on finding the minimum energy for different lattice parameters of silicon).

In [ ]:
from ase import io

# create a list for atomic distances to be tested
distances = np.arange(_________) # try from 0.3 to 2.3 angstroms

# find the second hydrogen in the "atomic positions" line in the input file
distance_index = qe_input.find('_________') + _________

total_energies = []
for distance in distances:
  # update the input file with the new atomic distance
  new_input_file = qe_input[:_________] + "%.2f" % _________ + qe_input[_________+_________:]

  # overwrite the input file
  with open("_________", "w") as f:
    f.write(_________)

  ! cat _________
  ! /content/q-e/bin/_________  -in _________ > _________ # run the DFT input file

  output = io.read("/content/hydrogen/_________") # read the output file

  total_energies.append(output._________()) # record the calculated total energy

energy0 = min(_________)
print("The minimum energy is: ", _________, " eV")

Plot the energy variation over different atomic distances.

In [ ]:
fig,ax = plt.subplots()
ax.plot(_________, _________)
ax.set_xlabel('_________') # don't forget units!
ax.set_ylabel('_________')
plt.show()

Perform a `relax` calculation using `pw.x`. First, we create another input file. Since we are relaxing the structure, we need to allow the ions (the core of the hydrogen atoms) to move.

There are different algorithms that allow this to occur - we will use the BFGS algorithm. You can read more about the BFGS algorithm [here](https://paulino.princeton.edu/courses/cee8813H/2020/class_notes/4_9_20/Online4_2_Newton%20&%20Quasi-Newton_Algorithms.pdf).

In [ ]:
# create input and write it into the file h2.relax.in
relax_input = """
&control
  prefix='h2',
  calculation='_________',
  verbosity='high',
  pseudo_dir = '_________',
  outdir='_________'
  tstress=false,
  tprnfor=false,
  restart_mode='from_scratch',
/
&system
  occupations='smearing',
  smearing='_________', ! try using the Marzari-Vanderbilt-DeVita-Payne cold smearing (see PRL 82, 3296 (1999))
  degauss=0.01,
  ecutwfc = _________,
  ecutrho = _________,
  ibrav = 1,
  celldm(1) = 20,
  nat = _________,
  ntyp = _________,
/
&electrons
  conv_thr = 1e-6,
/
&ions
  ion_dynamics = '_________',
/
ATOMIC_SPECIES
 _________  1.00784  _________
ATOMIC_POSITIONS (angstrom)
 _________ _________ _________ _________
 _________ _________ _________ _________
K_POINTS (automatic)
  1 1 1 0 0 0
"""

with open("_________", "w") as f:
    f.write(_________)

# print the content of the input file (under Linux)
! cat _________

Now run the `relax` calculation with `pw.x` to produce h2.relax.out.

In [ ]:
! _________  -in _________ > _________ # run the DFT input file
! cat _________

In [ ]:
output = io.read("/content/hydrogen/_________") # read the output file
print("The total energy is: ", output._________(), " eV")

final_h1 = output.get_positions()[_________]
final_h2 = output.get_positions()[_________]
final_distance = np.linalg.norm(_________)
# final_distance = output.get_all_distances()[0][1]
print("The interatomic distance is ", _________, " angstroms")
print("Forces acting on atoms ", output._________(), " eV/Angstrom")

- How do these results compare to the potential energy surface?
- Does it change the result if you start from a different point?
- When does it apply?

### Step 2: compute forces on displaced atoms.

For $H_2$ we only need to displace one atom in the direction along the bond, since this diatomic molecule has only this degree of freedom.

Something to think about: what about the $H_2O$ molecule instead? (We will explore this in the `6_QE_Vibrational_Modes_H2O` exercise.

In [ ]:
from ase.io import read
from ase import Atoms

# create a base input, where we can replace one of the atomic positions
qe_input = """
&control
  prefix='h2',
  calculation='_________',
  verbosity='high',
  pseudo_dir = '_________',
  outdir='_________'
  tstress=false,
  tprnfor=_________, ! need to set this to true to get forces
  restart_mode='from_scratch',
/
&system
  occupations='_________',
  smearing='_________',
  degauss=_________,
  ecutwfc = _________,
  ecutrho = _________,
  assume_isolated='_________', ! assume clusters are isolated: see doi:10.1063/1.477923
  ibrav = 1,
  celldm(1) = 20,
  nat = _________,
  ntyp = _________,
/
&electrons
  conv_thr = 1.0e-18,
/
ATOMIC_SPECIES
 _________  _________  _________
ATOMIC_POSITIONS (angstrom)
 _________ _________ _________ _________
 _________ _________ _________ _________
K_POINTS (automatic)
  1 1 1 0 0 0
"""

Replace of of the atomic positions with the desired distance for which we want to find the forces.

In [ ]:
d0 = _________ # relaxed distance, in angstroms
delta = _________ # displaced distance from relaxed positions, in angstroms
new_distance = _________ # total distance, in angstroms

# find the second hydrogen in the "atomic positions" line in the input file
distance_index = _________.find('_________') + _________

# update the input file with the new atomic distance
new_input_file = _________[:_________] + "%.5f" % _________ + _________[_________+_________:]

# overwrite the input file
with open("_________", "w") as f:
  f.write(_________)

! cat _________

In [ ]:
! _________  -in _________ > _________ # run the DFT input file

displaced_h2 = io._________("_________") # read the output file

forces = _________._________() # extract force
print("Forces acting on atoms: ", _________, " eV/Angstrom")

In [ ]:
plt.plot(_________, total_energies)
plt.plot(_________, _________, marker='o', c='black') # mark the minimum energy and distance
plt.plot(_________, _________._________(), marker='o', c='red') # mark the displaced energy and distance
plt.xlim(0.2,2.0)
plt.ylabel("_________") # don't forget units!
plt.xlabel("_________")

### Step 3: calculate the _force constant_

The force constant _C_ of a material is defined as the second order derivate with respect to atomic positions of the total energy.

For the $H_2$ molecule this means:
$
C = \frac{\partial^2 E}{\partial d^2}
= - \frac{\partial F}{\partial d}
= - \lim_{\delta d \rightarrow 0} \frac{F(d_0+\delta d) -F(d_0)}{\delta d}
\approx - \frac{F(d_0+\Delta d) -F(d_0)}{\Delta d}
\approx - \frac{F(d_0+\Delta d)}{\Delta d}
$

In [ ]:
d0 = _________ # angstroms
delta = 0.0025 # angstroms
new_distance = _________ # angstroms

# find the second hydrogen in the "atomic positions" line in the input file
distance_index = _________._________('_________') + _________

# update the input file with the new atomic distance
new_input_file = _________[:_________] + "%.5f" % _________ + _________[_________+_________:]

# overwrite the input file
with open("_________", "w") as f:
  f.write(_________)

! _________  -in _________ > _________ # run the DFT input file

displaced_h2 = io._________("_________") # read the output file

In [ ]:
forces_1 = displaced_h2._________() # extract force
force_constant = - _________[_________][_________] / _________ # apply simplified force constant equation
print("The force constant is :", _________, " eV/Angstrom^2")

### Step 4: calculate the vibrational _frequency_

In the classic harmonic model, atoms are subjected to the dynamics described by: $F = m \ddot{d} = - C~d$

The time solution is: $d(t) = A \sin(\omega t + \phi)$, where $\omega = \sqrt{\frac{C}{m}}$.

Let's calculae the oscillation frequency $\omega$ for the $H_2$ molecule!

In [ ]:
mass = 1 # atomic mass units
frequency = np._________(_________ / _________)
print("The oscillation frequency is: ", _________, " (eV/(Angstrom^2 * AMU))^(1/2)")

Let's convert the angold frequency units to THz

In [ ]:
ev_to_joule = 1.60217733e-19
amu_to_kg = 1.6605402e-27
angstrom_to_m = 1.0e-10
hz_to_thz = 1.0e-12

conversion_factor = _________ * np._________(_________/_________) / _________ / (2.0*np._________)
print("The conversion is: ", _________)

In [ ]:
print("The oscillation frequency is: ", _________ * _________, " THz")

In [ ]:
thz_to_invcm = 33.3564095198152
print("The oscillation frequency is: ", _________ * _________ * _________, " cm^(-1)")

## $H_2$ vibrational mode: density-functional perturbation theory

### Step 1: compute the ground-state using DFT

This step should account also for the structural optimization, meaning forces on all atoms and stress tensor should be close to zero.

A DFT calculation is then needed to provide the ground-state charge density for the DFPT calculation.

In [ ]:
# update the input file with the relaxed atomic distance
new_input_file = _________[:_________] + "%.5f" % _________ + _________[_________+_________:]

# overwrite the input file
with open("_________", "w") as f:
  f.write(_________)

! _________  -in _________ > _________ # run the DFT input file

ground_state = io._________("_________") # read the output file

ground_state_energy = _________._________() # eV
print('The ground state energy is ', _________, ' eV')

### Step 2: compute the _dynamical matrix_ via DFPT

Run the density-functional perturbation theory calculation to compute the dynamical matrix.

Define the inputs using plain string text and write to `h2.ph.in`.

In [ ]:
ph_inputs = """
&INPUTPH
  tr2_ph=1.0e-21,
  prefix='h2',
  verbosity='high',
  ldisp=.false.
  qplot=.false.
  alpha_mix(1)=0.4
/
  0.0, 0.0, 0.0
"""

with open('_________', 'w+') as handle:
    handle.write(_________)

Run the process using the `ph.x` executable to produce h2.ph.out.

In [ ]:
! _________  < _________ > _________ # run the ph.x executable

Read the results using ASE.

In [ ]:
from ase.io.espresso import read_espresso_ph

with open('_________', 'r') as handle:
    ph_results = read_espresso_ph(_________)

ph_results[1]['freqs'][-1] * thz_to_invcm

How does it compare with the value compute via finite differences?

Why are they different?

How does it compare with the experimental value 4161 $\mathrm{cm}^{-1}$?
(see _B. P. Stoicheff. 1957. HIGH RESOLUTION RAMAN SPECTROSCOPY OF GASES: IX. SPECTRA OF H2, HD, AND D2. Canadian Journal of Physics. 35(6): 730-741. https://doi.org/10.1139/p57-079_)

### Solution to discrepancy of finite differences and DFPT

The issue is that we did not use the correct formula! In fact, the calculation of frequencies involves a diagonalization of a (3N,3N) matrix. Look at the effect by diagonalizing the correct matrix, and then try to explain what should you modify in the previous section and why.

In [ ]:
from tabulate import tabulate

C = np.zeros((6,6))
C[2,5] = C[5,2] = C[2,2] = C[5,5] = _________

# Print the matrix as a nicely formatted table
print(tabulate(C, tablefmt="fancy_grid", floatfmt=".3f"))

Calculate the eigenvalues of the matrix using Numpy.

In [ ]:
np.sqrt(np.linalg.eigvals(C)) * thz_to_invcm * conversion_factor

How does it compare with the force constants previously computed?

What are the other eigenvalues corresponding to?

Why are they zero?